# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,1.94,87,100,7.33,RU,1726357077
1,1,kamaishi,39.2667,141.8833,26.77,76,83,3.99,JP,1726357165
2,2,ushuaia,-54.8000,-68.3000,5.81,65,75,5.14,AR,1726357066
3,3,pacasmayo,-7.4006,-79.5714,16.59,86,17,5.22,PE,1726357167
4,4,iqaluit,63.7506,-68.5145,6.85,75,20,2.06,CA,1726357169


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
                                 "Lng",
                                 "Lat",
                                 geo = True,
                                 tiles = "OSM",
                                 frame_width = 800,
                                 frame_height = 600,
                                 size = "Humidity",
                                 color = "City"
                                )
                                

# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_narrowed_down = city_data_df.loc[(city_data_df["Max Temp"]<=30) & (city_data_df["Max Temp"]>=15) 
                                     & (city_data_df["Humidity"]<=75) & (city_data_df["Cloudiness"]<=60)] 
                                    

# Drop any rows with null values
city_narrowed_down = city_narrowed_down.dropna(how='any')

# Display sample data
city_narrowed_down.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,hamilton,39.1834,-84.5333,29.69,45,0,5.14,US,1726357187
23,23,arkhipo-osipovka,44.3700,38.5200,21.51,48,0,5.04,RU,1726357191
35,35,al jawf,29.5000,38.7500,23.20,37,0,4.48,SA,1726357206
41,41,sovetskaya gavan',48.9723,140.2878,16.89,42,43,4.52,RU,1726357212
42,42,albany,42.6001,-73.9662,24.64,73,28,1.38,US,1726357213


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_narrowed_down[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
19,hamilton,US,39.1834,-84.5333,45,
23,arkhipo-osipovka,RU,44.3700,38.5200,48,
35,al jawf,SA,29.5000,38.7500,37,
41,sovetskaya gavan',RU,48.9723,140.2878,42,
42,albany,US,42.6001,-73.9662,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 1
radius = 5000

params = {
    "limit" : limit,
    "categories": f"accommodation.hotel",
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
     # get latitude, longitude from the DataFrame    
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
arkhipo-osipovka - nearest hotel: Санаторий "Архипо-Осиповка"
al jawf - nearest hotel: No hotel found
sovetskaya gavan' - nearest hotel: Гостинный Двор
albany - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
serhetabat - nearest hotel: No hotel found
ghormach - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
carnarvon - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
carthage - nearest hotel: Hôtel Amilcar
kharabali - nearest hotel: No hotel found
kerikeri - nearest hotel: Avalon Resort
hawaiian paradise park - nearest hotel: No hotel found
kalabo - nearest hotel: Golden Lodge Luxury Acomodation
kone - nearest hotel: Pacifik Appartels
nova sintra - nearest hotel: Residência Ka Dencho
geraldton - nearest hotel: Ocean Centre Hotel
nguigmi - nearest hotel: Guest PAM
lata - nearest hotel: No hotel found
arar - n

,City,Country,Lat,Lng,Humidity,Hotel Name
19,hamilton,US,39.1834,-84.5333,45,North Vista Manor
23,arkhipo-osipovka,RU,44.3700,38.5200,48,"Санаторий ""Архипо-Осиповка"""
35,al jawf,SA,29.5000,38.7500,37,No hotel found
41,sovetskaya gavan',RU,48.9723,140.2878,42,Гостинный Двор
42,albany,US,42.6001,-73.9662,73,No hotel found
...,...,...,...,...,...,...
547,poxoreo,BR,-15.8372,-54.3892,23,No hotel found
550,afaahiti,PF,-17.7500,-149.2833,75,Omati Lodge
556,urangan,AU,-25.3000,152.9000,54,Ramada
562,ovalle,CL,-30.5983,-71.2003,62,Hotel Roxy


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points(
                                 "Lng",
                                 "Lat",
                                 geo = True,
                                 tiles = "OSM",
                                 frame_width = 800,
                                 frame_height = 600,
                                 size = "Humidity",
                                 color = "City"
                                )

# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)